In [ ]:
import pandas as pd 
import numpy as np
import scipy
import matplotlib.pyplot as plt

In [ ]:
variables = [
    'cloud_cover', 'wind_speed', 'wind_gust', 'humidity',
    'pressure', 'global_radiation', 'precipitation', 'sunshine', 
    'temp_mean', 'temp_min', 'temp_max'
]

# Choose a variable to run
var_name = 'temp_max'

df_bilt = pd.read_csv('Data/de_bilt_weather.csv')

df_bilt['days'] = df_bilt['days'] - df_bilt['days'].min()

train_range = 365
test_range = 10

def fit_lrm(x, alfa, beta):
    return alfa + beta * x

def lrm(x, alfa, beta, sigma):
    return alfa + beta * x + np.random.normal(0, sigma)

#Rows from index 0 up to index 364 for training
x_train = df_bilt[var_name][:train_range]
#Rows from index 1 up to index 365, shifting by one day.
y_train = df_bilt[var_name][1:train_range+1]     

params, _ = scipy.optimize.curve_fit(fit_lrm, x_train, y_train)
alfa, beta = params

residuals = y_train - fit_lrm(x_train, alfa, beta)
std = np.sqrt(np.sum(residuals**2) / (len(x_train) - 2))

#starting point for forecasting future values.
start_value = df_bilt[var_name][train_range - 1]  # index 364

all_trajectories = []
for _ in range(1000):
    trajectory = []
    current_value = start_value
    # Predict for test_range days ahead: day 365 to 374
    for _ in range(test_range):
        next_value = lrm(current_value, alfa, beta, std)
        trajectory.append(next_value)
        current_value = next_value
    all_trajectories.append(trajectory)

mean_trajectory = np.mean(all_trajectories, axis=0)
observed_test = df_bilt[var_name][train_range:train_range+test_range].values
obs = observed_test
pred = mean_trajectory

mae = np.mean(np.abs(obs - pred))
mse = np.mean((obs - pred)**2)
rmse = np.sqrt(mse)

In [ ]:
#Get the test days (same indexing as VAR model)
days_test = df_bilt['days'].iloc[train_range:train_range+test_range].values

#Create a df for the LRM predictions
df_lrm = pd.DataFrame({
    "days": days_test,
    f"actual_{var_name}": observed_test,
    f"lrm_forecast_{var_name}": mean_trajectory
})

#Save LRM predictions to CSV file
df_lrm.to_csv("Testing/LRM_Data", index=False)

In [ ]:
#Predicted values of training set
y_pred_train = fit_lrm(x_train, alfa, beta)

#Finding total sum of squares and residual sum of squares
ss_total = np.sum((y_train - np.mean(y_train))**2)
ss_res = np.sum((y_train - y_pred_train)**2)

#Calculate R^2
r2 = 1 - (ss_res / ss_total)